# AutoFIS code experimenting

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
# data processing
import pandas as pd
import numpy as np

from main.autoFIS.autoFIS.fuzzification import Fuzzification
from main.autoFIS.autoFIS.formulation import Formulation
from main.autoFIS.autoFIS.association import Association
from main.autoFIS.autoFIS.aggregation import Aggregation
from main.autoFIS.autoFIS.decision import Decision

from sklearn.preprocessing import OneHotEncoder
from joblib import dump, load

In [3]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.metrics import accuracy_score

## Importing benchmark dataset Iris

In [4]:
iris = datasets.load_iris()
df_iris = pd.DataFrame(iris['data'])
df_iris['target'] = iris['target']

X = iris['data']
y = iris['target']

X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.30, random_state=42)

Parameters are set by the user

In [5]:
categorical_attributes = [False, False, False, False]
triangle_format = 'normal'  # "tukey", "normal"
n_fuzzy_sets = 5  # 3, 5, 7
enable_negation = False

# -------------------------
# Formulation parameters
# -------------------------
premise_max_size = 2
t_norm = 'prod'  # "min", "prod"

# Area filter parameters:
criteria_support = "cardinalidade_relativa"  # "cardinalidade_relativa", "frequencia_relativa"
area_threshold = 0.05

enable_pcd_premises_base = True
enable_pcd_premises_derived = True

# Overlapping filter parameters:
enable_similarity_premises_bases = True
enable_similarity_premises_derived = True
threshold_similarity = 0.95

association_method = "MQR"  # "MQR", "PMQR", "CD", "PCD", "freq_max"

aggregation_method = "MQR"  # "MQR", "PMQR", "CD", "PCD", "freq_max"

Fuzzification

In [6]:
fuzzification_params = {
    'triangle_format' : 'normal',
    'n_fuzzy_sets': n_fuzzy_sets,
    'enable_negation': enable_negation
}

fuzzifier = Fuzzification(**fuzzification_params)
fuzzifier.fit(X_train,is_categorical = categorical_attributes)
# TODO: put fuzzy_premises and num_fuzzy_premises to be calculated INSIDE the formulation step. Not in the fuzzification step. this is wrong.
uX,fuzzy_premises,num_fuzzy_premises = fuzzifier.transform(X_train)

Formulation

In [7]:
# TODO: refactor this part when building the AutoFIS class
y_categories = np.unique(y_train)
y_encoder = OneHotEncoder(categories = [y_categories], drop='if_binary')
y_train_one_hot = y_encoder.fit_transform(y_train.reshape(-1, 1)).toarray()
number_classes = y_train_one_hot.shape[1]
percentage_of_classes = y_train_one_hot.sum(axis = 0) / y_train_one_hot.shape[0]

In [8]:
formulation_params = {

    'antecedents_by_attribute': fuzzy_premises,
    'num_of_antecedents_by_attribute': num_fuzzy_premises,
    
    'attribute_is_binary': fuzzifier.fuzzy_params['is_binary'],
    'ux': uX, 
    'target_class': y_train_one_hot,
    'enable_negation': enable_negation,
    't_norm':t_norm,
    'premise_max_size' : premise_max_size,
    'criteria_support' : criteria_support,  # 'cardinalidade relativa', 'frequencia relativa'
    'threshold_support' : area_threshold,  # tolerancia da area
    'enable_similarity_premises_bases' : enable_similarity_premises_bases,
    'enable_similarity_premises_derived' : enable_similarity_premises_derived,
    'threshold_similarity' : threshold_similarity,
    'enable_pcd_premises_base' : enable_pcd_premises_base,
    'enable_pcd_premises_derived' : enable_pcd_premises_derived
}

formulator = Formulation(**formulation_params)
tree = formulator.generate_premises()

#TODO: Colocar essas mensagens dentro de alguma etapa de validação de premissas.
status = [False if not i[0] else True for i in tree] # checa se tem alguma ordem de premisas vazia
sum_status = sum(status)
if sum_status != len(tree): 
    if sum_status == 0:
        raise ValueError("Error in Formulation Module. Any premise survived. "
                            "Sorry, you can not continue in the next stage."
                            "\nTry to change the configuration")
    else:
        # TODO: Consertar isso. Acho que ele filtra só as ordens de premissas que estão preenchidas
        arb = [i for i in tree if i[0]]
        tree, arb = arb, tree

print('Done with Formulation...')
for i,values in enumerate(tree):
    print('Depth level ' + str(i + 1) + ': ' + str(tree[i][1].shape[1] ))

Done with Formulation...
Depth level 1: 18
Depth level 2: 31


Association

In [9]:
#TODO: Se sobrar tempo, fazer class polymorphismo nos métodos de agregação

associator = Association(tree, y_train_one_hot)
association_rules = associator.build_association_rules(association_method)

#TODO: Colocar essas mensagens dentro de alguma etapa de validação de premissas.
status = [0 if not i[0] else 1 for i in association_rules]
if sum(status) != number_classes:
    raise ValueError("Error in Association Module. Some classes did not get premises. "
                     "\nTry to change the configuration")
print('Done with Association...')
print('')
print('Rules per class:')

for index,i in enumerate(association_rules):
    print('class ', index, ': ', i[0])

Done with Association...

Rules per class:
class  0 :  [(1,), (10,), (15,), (10, 15)]
class  1 :  [(12,), (17,), (12, 17)]
class  2 :  [(13,), (18,), (19,), (13, 18), (13, 19)]


Aggregation

In [10]:
#TODO: Se sobrar tempo, fazer class polymorphismo nos métodos de agregação
#TODO: Se sobrar tempo [2], remover import de auxfunc e deixar tudo dentro de uma classe só. A menos q esteja sendo usado em outro lugar a mesma função.
aggregator = Aggregation(association_rules)
aggregation_rules,estimation_classes  = aggregator.aggregate_rules(y_train_one_hot, aggregation_method)

#TODO: Colocar essas mensagens dentro de alguma etapa de validação de premissas.

status = [0 if not i[0] else 1 for i in aggregation_rules]
if sum(status) != number_classes:
    raise ValueError("Error in Aggregation Module. Some classes did not get premises. "
                        "Sorry, you can not continue in the next stage."
                        "\nTry to change the configuration")
print('Done with Aggregation...')
print('')

final_premises_classes = []
for index,i in enumerate(aggregation_rules):
    print("Premises of Class " + str(index) + ": " + str(i[0]))
    final_premises_classes.append(i[0])
    print("weights: " + str(i[1].T))
    print('')

Done with Aggregation...

Premises of Class 0: [(1,), (10,), (15,)]
weights: [[0.02410548 0.62684905 0.34904547]]

Premises of Class 1: [(12,), (17,)]
weights: [[0.40570462 0.5942954 ]]

Premises of Class 2: [(13,), (18,), (19,)]
weights: [[0.44368515 0.44333386 0.11298105]]



Decision

In [11]:
decision_maker = Decision(aggregation_rules, percentage_of_classes)
y_train_prediction = decision_maker.predict(uX,t_norm)
y_train_prediction = y_encoder.inverse_transform(y_train_prediction)

In [12]:
print('train: ')
accuracy_score(y_train_prediction, y_train)

train: 


0.9619047619047619

Test

In [13]:
ux_test,_,_ = fuzzifier.transform(X_test)
# y_test_one_hot = y_encoder.transform(y_test.reshape(-1, 1)).toarray()

y_test_pred = decision_maker.predict(ux_test,t_norm)
y_test_pred = y_encoder.inverse_transform(y_test_pred)

In [14]:
print('test: ')
accuracy_score(y_test_pred, y_test)

test: 


0.9777777777777777